In [1]:
!pip install pyspark py4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=eda5a59a5dd3b1d817824f6c7c71ed52d2ff33bcb4ca106e6e186b2e8ab03f3c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("App").getOrCreate()

In [6]:
data = [
    (20124 ,'2020-01-10'),
    (40133 ,'2020-01-15'),
    (65005 ,'2020-01-20'),
    (30005 ,'2020-02-08'),
    (35015 ,'2020-02-19'),
    (15015 ,'2020-03-03'),
    (35035 ,'2020-03-10'),
    (49099 ,'2020-03-14'),
    (84045 ,'2020-03-20'),
    (100106 ,'2020-03-31'),
    (17015 ,'2020-04-04'),
    (36035 ,'2020-04-11'),
    (50099 ,'2020-04-13'),
    (87045 ,'2020-04-22'),
    (101101 ,'2020-04-30'),
    (40015 ,'2020-05-01'),
    (54035 ,'2020-05-09'),
    (71099 ,'2020-05-14'),
    (82045 ,'2020-05-21'),
    (90103 ,'2020-05-25'),
    (99103 ,'2020-05-31'),
    (11015 ,'2020-06-03'),
    (28035 ,'2020-06-10'),
    (38099 ,'2020-06-14'),
    (45045 ,'2020-06-20'),
    (36033 ,'2020-07-09'),
    (40011 ,'2020-07-23'),
    (25001 ,'2020-08-12'),
    (29990 ,'2020-08-26'),
    (20112 ,'2020-09-04'),
    (43991 ,'2020-09-18'),
    (51002 ,'2020-09-29'),
    (26587 ,'2020-10-25'),
    (11000 ,'2020-11-07'),
    (35002 ,'2020-11-16'),
    (56010 ,'2020-11-28'),
    (15099 ,'2020-12-02'),
    (38042 ,'2020-12-11'),
    (73030 ,'2020-12-26')
]

schema = "cases_reported int , dates string"
df = spark.createDataFrame(data= data , schema = schema)
df.show(5)


+--------------+----------+
|cases_reported|     dates|
+--------------+----------+
|         20124|2020-01-10|
|         40133|2020-01-15|
|         65005|2020-01-20|
|         30005|2020-02-08|
|         35015|2020-02-19|
+--------------+----------+
only showing top 5 rows



In [7]:
df.printSchema()

root
 |-- cases_reported: integer (nullable = true)
 |-- dates: string (nullable = true)



In [9]:
#convert the dates to date format. We use to_date function.

df_date = df.withColumn("dates", to_date("dates", "yyyy-MM-dd"))
df_date.show(5)

+--------------+----------+
|cases_reported|     dates|
+--------------+----------+
|         20124|2020-01-10|
|         40133|2020-01-15|
|         65005|2020-01-20|
|         30005|2020-02-08|
|         35015|2020-02-19|
+--------------+----------+
only showing top 5 rows



In [10]:
# new schema
df_date.printSchema()

root
 |-- cases_reported: integer (nullable = true)
 |-- dates: date (nullable = true)



In [13]:
df_group = df_date.groupBy(month(col("dates")).alias("Months")).agg(sum("cases_reported").alias("Monthly_Cases"))
df_group.show(5)

+------+-------------+
|Months|Monthly_Cases|
+------+-------------+
|     1|       125262|
|     3|       283300|
|     5|       436400|
|     4|       291295|
|     2|        65020|
+------+-------------+
only showing top 5 rows



In [17]:
#Finding the cumulative_cases column
from pyspark.sql import Window
df_cum = df_group.withColumn("cumulative_cases", sum(col("Monthly_Cases")).over(Window.orderBy(col("Months"))))
df_cum.show(5)

+------+-------------+----------------+
|Months|Monthly_Cases|cumulative_cases|
+------+-------------+----------------+
|     1|       125262|          125262|
|     2|        65020|          190282|
|     3|       283300|          473582|
|     4|       291295|          764877|
|     5|       436400|         1201277|
+------+-------------+----------------+
only showing top 5 rows



In [25]:
# prior months cumulative cases

df_lag = df_cum.withColumn("Prior_month_cumulative", lag(col("cumulative_cases"),1,0).over(Window.orderBy(col("Months"))))
df_lag.show(5)

+------+-------------+----------------+----------------------+
|Months|Monthly_Cases|cumulative_cases|Prior_month_cumulative|
+------+-------------+----------------+----------------------+
|     1|       125262|          125262|                     0|
|     2|        65020|          190282|                125262|
|     3|       283300|          473582|                190282|
|     4|       291295|          764877|                473582|
|     5|       436400|         1201277|                764877|
+------+-------------+----------------+----------------------+
only showing top 5 rows



In [26]:
# Applying formula

final_df = df_lag.withColumn("Percentage_increase", round(col("monthly_cases")*100/col("Prior_month_cumulative"),1))
final_df.show(5)

+------+-------------+----------------+----------------------+-------------------+
|Months|Monthly_Cases|cumulative_cases|Prior_month_cumulative|Percentage_increase|
+------+-------------+----------------+----------------------+-------------------+
|     1|       125262|          125262|                     0|               NULL|
|     2|        65020|          190282|                125262|               51.9|
|     3|       283300|          473582|                190282|              148.9|
|     4|       291295|          764877|                473582|               61.5|
|     5|       436400|         1201277|                764877|               57.1|
+------+-------------+----------------+----------------------+-------------------+
only showing top 5 rows



In [27]:
# Remove null
final_df = df_lag.withColumn("Percentage_increase", when(col("Months") > 1, round(col("monthly_cases")*100/col("Prior_month_cumulative"),1)).otherwise("-"))
final_df.show(5)

+------+-------------+----------------+----------------------+-------------------+
|Months|Monthly_Cases|cumulative_cases|Prior_month_cumulative|Percentage_increase|
+------+-------------+----------------+----------------------+-------------------+
|     1|       125262|          125262|                     0|                  -|
|     2|        65020|          190282|                125262|               51.9|
|     3|       283300|          473582|                190282|              148.9|
|     4|       291295|          764877|                473582|               61.5|
|     5|       436400|         1201277|                764877|               57.1|
+------+-------------+----------------+----------------------+-------------------+
only showing top 5 rows

